In [1]:
import os
import numpy as np
import pandas as pd
import h5py
from matplotlib import pyplot as plt
%matplotlib inline
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.metrics import roc_auc_score
import keras #Change later once I have metrics done/
import my_callbacks
from keras.utils import np_utils
import my_metrics


Using TensorFlow backend.


In [2]:
bg_files = ['/afs/cern.ch/user/f/fsiroky/public/JetHC_Cbackground.h5', '/afs/cern.ch/user/f/fsiroky/public/JetHC_D_background.h5','/afs/cern.ch/user/f/fsiroky/public/JetHC_E_background.h5','/afs/cern.ch/user/f/fsiroky/public/JetHC_F_background.h5','/afs/cern.ch/user/f/fsiroky/public/JetHC_G_background.h5','/afs/cern.ch/user/f/fsiroky/public/JetHC_H_background.h5']
bg_jets = ["jet_C_background", "jet_D_background","jet_E_background","jet_F_background","jet_G_background","jet_H_background"]
sig_files = ['/afs/cern.ch/user/f/fsiroky/public/JetHC_Csignal.h5', '/afs/cern.ch/user/f/fsiroky/public/JetHC_D_signal.h5', '/afs/cern.ch/user/f/fsiroky/public/JetHC_E_signal.h5', '/afs/cern.ch/user/f/fsiroky/public/JetHC_F_signal.h5', '/afs/cern.ch/user/f/fsiroky/public/JetHC_G_signal.h5', '/afs/cern.ch/user/f/fsiroky/public/JetHC_H_signal.h5']
sig_jets = ["jet_C_signal", "jet_D_signal", "jet_E_signal", "jet_F_signal", "jet_G_signal", "jet_H_signal"]

def get_jets(bg_files, bg_jets, sig_files, sig_jets, ratio=1.0):
    good_jets=np.empty([0,2802])
    bad_jets=np.empty([0,2802])
    np.random.seed(24)
    for i in range(len(bg_files)):
        bg_jetfile = h5py.File(bg_files[i],'r')
        bg_jet = bg_jetfile[bg_jets[i]][:]
        sig_jetfile = h5py.File(sig_files[i],'r')
        sig_jet = sig_jetfile[sig_jets[i]][:]

        bad_jets = np.concatenate((bad_jets, bg_jet), axis=0)

        ratio_abs=ratio*len(bg_jet)/len(sig_jet)

        mask = np.random.rand(len(sig_jet))<ratio_abs
        good_jets = np.concatenate((good_jets, sig_jet[mask]), axis=0)
        #change the way of saving good_jets if we want to normalize C,D,... specifically


        print(np.sum(mask), len(bg_jet), len(good_jets), ratio_abs)
        
    return good_jets, bad_jets

def get_train_test_split(good_jets, bad_jets):
    x_train = np.concatenate((good_jets,bad_jets), axis=0)
#     print(x_train)
    scaler = Normalizer()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)

    y_train = np.zeros(len(x_train))
    y_train[0:len(good_jets)]=1
    mask = shuffle((np.arange(len(x_train))), random_state=42)

    x_train = x_train[mask]
    y_train = y_train[mask]

    x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.15, random_state=42)
    return x_train, x_test, y_train, y_test

In [3]:
%%time

scores = []
ratios = [20]
xy_ratio=[]

for e in ratios:
    good_jets, bad_jets = get_jets(bg_files, bg_jets, sig_files, sig_jets, ratio=e)
    x_train, x_test, y_train, y_test = get_train_test_split(good_jets, bad_jets)
    x_train, x_val, y_train, y_val   = train_test_split(x_train, y_train, test_size=0.15, random_state=1)

    model = Sequential()
    model.add(Dense(2802, input_dim=2802, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1000, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='Adam',
                  metrics=['accuracy',my_metrics.f1,my_metrics.mcor])
#     callbacks=[roc_callback(x_train=x_train,y_train=y_train,x_test=x_test,y_test=y_test)]
#     histories = my_callbacks.Histories()
#     class_weight = {0 : 1.,
#     1: 10.}
    model.fit(x_train, y_train, 
#               validation_split=0.15,
              validation_data=(x_val,y_val),
              epochs=20,
              batch_size=128)
#              class_weight = class_weight)
#              callbacks=[histories],
#              validation_data=(x_test, y_test))
#     print("callbacks ",callbacks[0].on_epoch_end(0))
#     print("Losses are ",histories.losses)
#     print("Aucs are ",histories.aucs)

    score = model.evaluate(x_test, y_test, batch_size=128)
    y_pred = model.predict(x_test)
    print()
    print("ROC AUC SCORE is",roc_auc_score(y_test, y_pred))



    scores.append(score)
    acc = [e[1] for e in scores]
    print()
    print(acc)
    xy_ratio.append((len(good_jets)/(len(bad_jets)+len(good_jets))))
    print(xy_ratio)
    print("xxxxxxxxxx")
    

# 3401 3297

9263 459 9263 0.544128978720882
1925 97 11188 0.0961252601327916
5369 270 16557 0.5630865484880083
1423 70 17980 0.07927070947284978
3863 193 21843 0.1003379256563556
35362 2208 57205 1.2487981449013066
Train on 43712 samples, validate on 7714 samples
Epoch 1/20
43712/43712 [==============================] - 101s - loss: 0.2098 - acc: 0.9437 - f1: 0.9696 - mcor: 1.3579e-04 - val_loss: 0.2142 - val_acc: 0.9398 - val_f1: 0.9689 - val_mcor: 0.0000e+00

In [4]:
whos
# acc = [e[1] for e in scores]
# plt.plot(ratios, acc, '*')
# acc

NameError: name 'whos' is not defined

In [ ]:
def foo(p):
    print(p)
    p = 333
    print(p)
    
foo(777)

In [ ]:
def normalize(h):
    return h / np.sum(h)

def bhattacharyya(h1, h2):
  '''Calculates the Bhattacharyya distance of two histograms.'''

  return 1 - np.sum(np.sqrt(np.multiply(normalize(h1), normalize(h2))))

In [ ]:
def plot_variables(sigVals, backVals, var_list, save_figs_path=None, var_range_to_plot=None):
    if var_range_to_plot is None:
        var_range_to_plot = range(len(var_list))
    
    labels = []
    for i in range(len(var_list)):
            for suffix in ['Mean','RMS','Q1','Q2','Q3','Q4','Q5']:
                labels.append(var_list[i]+'_'+suffix)
    
    for var_id in var_range_to_plot: 
        print(var_list[var_id])
        fig = plt.figure(figsize=(15, 7))
        for i in range(7):
            plt.subplot(2, 4, i + 1)    
            index = var_id*7+i
            minimum = np.min( [np.min(backVals[:,index]), np.min(sigVals[:,index])] )
            maximum = np.max( [np.max(backVals[:,index]), np.max(sigVals[:,index])] )
#             back_dist = plt.hist(backVals[:,index], 100, range=(minimum,maximum), facecolor='red', alpha=0.5, label="background", normed = True)
#             sig_dist = plt.hist(sigVals[:,index], 100, range=(minimum,maximum), facecolor='blue', alpha=0.5, label="normal", normed = True)
            back_dist = plt.hist(backVals[:,index], 100, range=(minimum,maximum), facecolor='red', alpha=0.5, label="background", normed = False)
            sig_dist = plt.hist(sigVals[:,index], 100, range=(minimum,maximum), facecolor='blue', alpha=0.5, label="normal", normed = False)

            bhat = bhattacharyya(back_dist[0], sig_dist[0])
#             ks = ks_2samp(back_dist[0], sig_dist[0])
#             rms = rms_dist(back_dist[0], sig_dist[0])
#             rms2 = rms_dist2(back_dist[0], sig_dist[0])
#             distance.
#             plt.plot([],[], '.', label="Bhatta {:.6f} \nK-S  s={:.6f} \n      p={:.6f}".format(bhat,ks[0],ks[1]))
#             plt.plot([],[], '.', label="Bhatta {:.6f} \nrms= {:.6f} \nrms2={:.6f}".format(bhat,rms,rms2))
            
            plt.plot([],[], '.', label="Bhatta {:.4f}".format(bhat))

            plt.legend(loc="upper right")
            plt.title(labels[index],fontsize=16)

        ax = plt.subplot(2,4,8)
        plt.plot([0,1],[0,1],'.')
    #     plt.title('notes',fontsize=16)
        plt.text(0, 0.9, 'number of events', fontsize=15)
        plt.text(0, 0.7, 'background '+str(backVals.shape[0]), fontsize=15)
        plt.text(0, 0.5, 'normal '+str(sigVals.shape[0]), fontsize=15)
        plt.setp(ax.get_xticklabels(),visible=False)
        plt.setp(ax.get_yticklabels(),visible=False)

        plt.tight_layout()
        # setting the global title and pushing the subplots down accordingly 
        st = fig.suptitle(var_list[var_id], fontsize=25)
        st.set_y(0.95)
        fig.subplots_adjust(top=0.85)
        
        if save_figs_path is not None:
            plt.savefig(save_figs_path+'/'+var_list[var_id]+'.png', dpi=fig.dpi)
    print('done plot_variables')

In [ ]:

plot_variables(good_jets, bad_jets, var_names_reduced5, var_range_to_plot=[44,58])

In [ ]:
sig_jets.shape

In [ ]:
whos